# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [80]:
from xml.etree import ElementTree as ET
import re
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [6]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [6]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.iter('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [7]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()


In [171]:
str_init = ""      #initialize an empty string
for element in root.findall('country'):   #iterate through all countries
    
    if element.find('infant_mortality') == None:
        
        pass
    # If element.find('infant_mortality').text is NOT null, extract the infant_mortality and country name as text and
    #add it to str_init for every country
    else:
        infant_mort = element.find('infant_mortality').text
        Country = element.find('name').text
        str_init = str_init + Country + infant_mort
        
list_im = re.split(r'(\d+\.*\d*)',str_init )  #split the string using regular expression which splits country and infant mortalities
dict_im = dict(zip(list_im[::2],list_im[1::2])) # create two separate lists for infant_mortality values and country names, zip them together, and then create a dictionary
df = pd.DataFrame(list(dict_im.items()), columns = ['Country','Infant_Mortality'],dtype = 'float') # construct a dataframe using the new dictionary
df.sort_values('Infant_Mortality',ascending = False).head(10) #sort the dataframe by infant mortality.

,Country,Infant_Mortality
178,Western Sahara,145.82
48,Afghanistan,117.23
173,Mali,104.34
210,Somalia,100.14
197,Central African Republic,92.86
214,Guinea-Bissau,90.92
198,Chad,90.30
176,Niger,86.27
179,Angola,79.99
185,Burkina Faso,76.80


In [174]:
city_pop_string = ''
city_pop_string2 = '' #initialize two empty strings

for country in root.findall('country'): #Loop through each country
    for city in country.iter('city'): #Loop through each city in each country
        if city.find('population') == None: #If city population is not specified, then pass
            pass
        else: #If city population is specified, find the city name and strip out all blank spaces, periods, commas. 
            
            city_name = city.find('name').text
            city_name = city_name.replace(' ', '').replace('-','').replace('.','').replace("'", "")
            for population in city.findall('population'): # For each city, find all population values and add it to city_pop_string
                city_pop_string += population.text+ " "
                city_pop_string += city_name+ ","

#Split the city_pop_string pair the latest population estimate with the city name
city_population_list = re.split(r'(\d+\s+\w+[\.,])', city_pop_string)

for element in city_population_list[1::2]: #Filter out old population estimates by indexing and loop through the list to add all elements of that list to city_pop_string2
    city_pop_string2 += element + " "

# Split city_pop_string_2 in order to separate population estimates and city name
city_final_lst = re.split(r'(\d+|\s+|[\.,])', city_pop_string2)

city_list = city_final_lst[4::8] # create a list consisting of only city names
latest_pop_lst = city_final_lst[1::8] #create a list consisting of only population estimates

city_pop_df = pd.DataFrame([city_list, latest_pop_lst]).unstack().unstack() #create a dataframe
city_pop_df.columns = ['City','Population']
city_pop_df['Population'] = city_pop_df['Population'].apply(lambda x: int(x)) #convert Population column to float dtype
city_pop_df.sort_values('Population', ascending = False).head(10) #sort the dataframe by population from largest to smallets

,City,Population
3693,Shanghai,22315474
3692,Shanghai,15758892
2550,Istanbul,13710512
4337,Delhi,12877470
4242,Mumbai,12442373
1489,Moskva,11979529
4241,Mumbai,11914398
3689,Beijing,11716620
1488,Moskva,11612885
8145,SãoPaulo,11152344


In [181]:
pop_str = ""
str_list_1 = "" #initialize two empty strings

for country in root.findall('country'): #loop through each country
    for pop in country.findall('population'): #loop through all population estimates for each country
        pop_str += pop.text + " " #Add all populations for each country to pop_str separated by a space
    pop_str += "." # Add a period when finished looping through a countrie's population

list_1 = re.split(r'(\d+\s+[\.,])',pop_str) # Split pop_str in order to separate latest population estimate from older estimates

for elem in list_1[1::2]: # Add all latest population estimates to a str_list_1
    str_list_1 += elem
    
list_2 = re.split(r'(\d+)', str_list_1)
recent_population_list = list_2[1::2] #Split list_2 via indexing to obtain a list of all recent population estimates for each country



In [182]:
master_str_eth = "" #intialize an empty string
j = 0

for country in root.findall('country'): #Loop through each country
    pop = country.find('name').text #extract country name
    pop_val = recent_population_list[j] #extract country population from recent_population_list
    j = j + 1
    for ethnic in country.iter('ethnicgroup'): #loop thorugh and extract all ethnic group population percent for each country
        list_eth = ethnic.items()[0][1]
        master_str_eth += list_eth + ethnic.text + "  " + pop + pop_val + "  " # add the extracted items to empty string
    
#split the master_str_eth string to separate out all the items
master_list_eth = re.split(r'(\d+\.*\d*)|\s{2}', master_str_eth)

eth_list = master_list_eth[2::8] #creates a list containing ethnic groups
percent_list = master_list_eth[1::8] #creates a list containing ethnic population distributions
country_list = master_list_eth[4::8] #creates a list containing all country names
pop_list = master_list_eth[5::8] #creates a list containing all country populations


df_2 = pd.DataFrame([eth_list, percent_list, country_list, pop_list]).unstack().unstack()#create a dataframe
df_3 = df_2[[2,0,1,3]]
df_3.columns = ['Country','Ethnicity','Percent','Population'] #rename columns 
df_3['Percent'] = df_3['Percent'].apply(lambda x: float(x))
df_3['Population'] = df_3['Population'].apply(lambda x: float(x))
df_3['Total'] = ((df_3.Percent)/100) * df_3.Population #calculate ethnic populations and add a "Total" column
df_3['Total'] = df_3['Total'].apply(lambda x: int(x)).round()

df_4 = df_3[['Ethnicity','Total']]
df_4.groupby('Ethnicity').sum().sort_values('Total',ascending = False).head(10)




,Total
Ethnicity,
Han Chinese,1245058800
Indo-Aryan,871815583
European,494872201
African,318325104
Dravidian,302713744
Mestizo,157734349
Bengali,146776916
Russian,131856989
Japanese,126534212


In [99]:
river_name = ""
river_country = ""
river_length = "" #initialize empty strings

for river in document.iter('river'): #loop through all rivers in the document
    river_name += river.find('name').text + "," #extract river name and add to river_name
    river_country += river.attrib['country'] + "," #extract the country/ies the river is located in and add to river_country
    if river.find('length') == None: 
        river_length += 'None'+","
    else: #if river length is not specified, add "None" to river_length
        river_length += river.find('length').text + "," #if river length is specified, extract the length and add to river_length
        
river_length_lst = river_length.split(',') #create separate lists containing river lengths, names, and country names
river_name_lst = river_name.split(',')
river_country_lst = river_country.split(',')

river_df = pd.DataFrame([river_length_lst[0:-1], river_name_lst[0:-1], river_country_lst[0:-1]]).unstack().unstack()
river_df.columns = ['River_Length','River_Name','Country'] # Create a dataframe and rename columns

river_df = river_df.replace('None', np.nan) #replace "None" with NaN values
river_df['River_Length'] = river_df['River_Length'].apply(lambda x: float(x))
river_df.sort_values('River_Length', ascending=False).head(1)

,River_Length,River_Name,Country
174,6448.0,Amazonas,CO BR PE


In [113]:
#Repeat steps used to solve river problem

lake_name = ""
lake_country = ""
lake_area = ""

for lake in document.iter('lake'):
    lake_name += lake.find('name').text + ","
    lake_country += lake.attrib['country'] + ","
    if lake.find('area') == None:
        lake_area+= 'None'+","
    else:
        lake_area += lake.find('area').text + ","
        
lake_area_lst = lake_area.split(',')

lake_name_lst = lake_name.split(',')
lake_country_lst = lake_country.split(',')



lake_df = pd.DataFrame([lake_area_lst[0:-1], lake_name_lst[0:-1], lake_country_lst[0:-1]]).unstack().unstack()

lake_df.columns = ['River_Length','River_Name','Country']

lake_df = lake_df.replace('None', np.nan)
lake_df['River_Length'] = lake_df['River_Length'].apply(lambda x: float(x))
lake_df
lake_df.sort_values('River_Length', ascending=False).head(1)

,River_Length,River_Name,Country
54,386400.0,Caspian Sea,R AZ KAZ IR TM


In [164]:
# Repeat steps used to solve river problems

airport_name = ""
airport_country = ""
airport_elevation = ""

for airport in document.iter('airport'):
    airport_name += airport.find('name').text + ","
    airport_country += airport.attrib['country'] + ","
    if airport.find('elevation').text == None:
        airport_elevation += 'None' + ","
    else:
        airport_elevation += airport.find('elevation').text + ","
        
airport_elevation_lst = airport_elevation.split(',')
airport_name_lst = airport_name.split(',')
airport_country_lst = airport_country.split(',')

airport_df = pd.DataFrame([airport_name_lst[0:-1], airport_elevation_lst[0:-1], airport_country_lst[0:-1]]).unstack().unstack()

airport_df.columns = ['Airport_Name','Airport_Elevation','Country']

airport_df = airport_df.replace('None', np.nan)
airport_df['Airport_Elevation'] = airport_df['Airport_Elevation'].apply(lambda x: float(x))
airport_df.sort_values('Airport_Elevation', ascending=False).head(1)

,Airport_Name,Airport_Elevation,Country
80,El Alto Intl,4063.0,BOL
